In [1]:
import PIL
import subprocess
import re
import os
import time
import io
import cv2
import requests
import bs4
import jieba
import jieba.analyse

In [2]:
def capture_img():
    subprocess.call('.\\adb\\adb shell /system/bin/screencap -p /sdcard/screenshot.png')
    time.sleep(0.1)
    subprocess.call('.\\adb\\adb pull /sdcard/screenshot.png %s' % './tmp/screenshot.png')
    time.sleep(0.1)
    
    return cv2.imread('./tmp/screenshot.png')

In [3]:
%run aip_key.py

In [4]:
from aip import AipOcr

client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

In [5]:
def get_text_in_region(aip_rst, region):
    s = ''
    if 'words_result' in aip_rst:
        words = []
        for p in aip_rst['words_result']:
            loc = p['location']
            if loc['top'] > region[0] and loc['top'] + loc['height'] < region[1] \
                and loc['left'] > region[2] and loc['left'] + loc['width'] < region[3]:
                if loc['left'] < 1080 / 2: # only get words that starts from left
                    words.append(p['words'])
        s = ''.join(words)
        
    if s != '':
        # remove heading, like 1. A. B.
        if re.match(r"^A|B|C|\d+\.|,|。|，", s) != None: 
            ss = re.split(r"\.|,|。|，",s)
            if len(ss) > 1:
                s = ''.join(ss[1:])
            
    return s

In [6]:
def ocr(im, regions):
    _, byte_arr = cv2.imencode('.jpg', im, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
    aip_rst = client.general(byte_arr.tobytes())
    return [get_text_in_region(aip_rst, region) for region in regions]

In [7]:
def go_search(s, engine='google'):
    if engine == 'google':
        os.system('start www.google.com/search?q=%s' % s)
    elif engine == 'baidu':
        os.system('start www.baidu.com/s?wd=%s' % s)

In [39]:
def search(question):
    rst = requests.get('http://www.baidu.com/s?wd=%s&rn=50' % question)
    rst.encoding = 'utf-8'
    search_rst = rst.text
    
    soup = bs4.BeautifulSoup(search_rst, 'html.parser')
    rst = soup.select('div#content_left')[0]
    return rst.text

def search_advanced(question_words, opt_words):
    '''
    Advanced search
    question_words: list of words of the question
    opt_words: list of list of words of the options
    '''
    all_opt_words = []
    
    for words in opt_words:
        all_opt_words += words
        
    rst = requests.get('http://www.baidu.com/s?q1=%s&q2=&q3=%s&q4=&rn=50' % (
        '+'.join(question_words),
        '+'.join(all_opt_words)))
    print('http://www.baidu.com/s?q1=%s&q2=&q3=%s&q4=&rn=50' % (
        '+'.join(question_words),
        '+'.join(all_opt_words)))
    rst.encoding = 'utf-8'
    search_rst = rst.text
    
    soup = bs4.BeautifulSoup(search_rst, 'html.parser')
    rst = soup.select('div#content_left')[0]
    return rst.text

In [13]:
help(list.extend)

Help on method_descriptor:

extend(...)
    L.extend(iterable) -> None -- extend list by appending elements from the iterable



In [9]:
def occur_count(s,words):
    n = 0
    for word in words:
        n += len(re.findall(word, s))
    return n

In [10]:
def keywords(question, opt_a, opt_b, opt_c):
    quest_words = jieba.analyse.extract_tags(question, allowPOS=['n','nr','ns','nt','nz','nl','ng'])
    opt_words = jieba.analyse.extract_tags(' '.join([opt_a,opt_b,opt_c]), allowPOS=['n','nr','ns','nt','nz','nl','ng'])
    words = quest_words + opt_words
    return words

In [51]:
def dump_question(question, opt_a, opt_b, opt_c):
    with open('questions.txt','a+') as f:
        f.write(','.join([question, opt_a, opt_b, opt_c]) + '\n')

In [52]:
def search_ans_from_pic(im):
    print('Parsing pic...')
    
    question, opt_a, opt_b, opt_c = ocr(im,
                   [(300,600,0,1080),
                   (655,815,0,1080),
                   (845,1005,0,1080),
                   (1035,1200,0,1080)])

    if question != '':
        quest_words = jieba.analyse.extract_tags(question)

        opts = [opt_a, opt_b, opt_c]
        opt_words = []
        for opt in opts:
#             opt_words.append(list(map(lambda p:p.word, filter(lambda p:p.flag.startswith('n'), 
#                                                  jieba.posseg.cut(opt)))))
            opt_words.append(jieba.analyse.extract_tags(opt))

#         rst = search(question)
#         time.sleep(0.1)
#         rst += search('%20'.join(quest_words))
        rst = search_advanced(quest_words, opt_words)

        opt_cnt = []
        for i, opt_w in enumerate(opt_words):
            opt_cnt.append(occur_count(rst, opt_w))


        ans = [('A',opt_a,opt_cnt[0]),
              ('B',opt_b,opt_cnt[1]),
              ('C',opt_c,opt_cnt[2])]
        
        

        print('question:', question)
        for p in sorted(ans, key=lambda p:p[2], reverse=True):
            print(p)

#         go_search(question)

        dump_question(question, opt_a, opt_b, opt_c)

    else:
        print('No question found!')

In [18]:
im = cv2.imread('./tmp/screenshot.png')
im = im[:1200,:,:]

In [53]:
search_ans_from_pic(im)

Parsing pic...
http://www.baidu.com/s?q1=海沟+最深+哪个+以下&q2=&q3=马里亚纳+海沟+波多黎各+海沟+海沟+菲律宾&q4=&rn=50
question: 以下哪个海沟最深?
('A', '马里亚纳海沟', 138)
('C', '菲律宾海沟', 135)
('B', '波多黎各海沟', 119)


In [ ]:
while True:
    ipt = input()
    if ipt == 'q':
        break
    else:
        im = capture_img()
        search_ans_from_pic(im[:1200,:,:])


Parsing pic...
question: 茅台酒原产于我国哪里?
('B', '山西省', 0)
('C', '河南省', 0)
('A', '贵州省', 4)

Parsing pic...
question: 我们常说“人心齐什么移"?
('A', '黄山', 0)
('C', '华山', 0)
('B', '泰山', 25)

Parsing pic...
question: 我和西瓜视频独家出品的脱口秀节目名称是?
('B', '《一郭烩》', 0)
('C', '《郭的秀》', 0)
('A', '《一郭汇》', 14)

Parsing pic...
question: 《哈利波特》的作者JK罗琳是哪国人?
('B', '美国', 0)
('C', '澳大利亚', 0)
('A', '英国', 10)

Parsing pic...
question: 歌歌曲《十年》的原唱是哪位歌手?
('A', '林俊杰', 0)
('B', '周杰伦', 0)
('C', '陈奕迅', 13)

Parsing pic...
question: 下列哪一部是鲁迅创作的文学作品?
('B', '《我的大学》', 0)
('C', '《家》', 0)
('A', '《祝福》', 1)

Parsing pic...
question: “2x-1=0”这是什么方程?
('A', '二元一次方程', 1)
('B', '一元二次方程', 1)
('C', '元一次方程', 1)

Parsing pic...
question: 现在以下哪一艺术作品没有馆藏于巴黎卢浮宫?
('A', '《蒙娜丽莎》', 3)
('B', '《断臂的维纳斯》', 3)
('C', '《最后的晚餐》', 4)

Parsing pic...
question: 标准大气压下下列哪种金属的熔点最高?
('A', '锡', 0)
('B', '铁', 0)
('C', '金', 0)
.
Parsing pic...
question: 我国在哪一年成功研制出第一颗原子弹?
('B', '1968年', 0)
('C', '1970年', 0)
('A', '1964年', 8)

Parsing pic...
question: 以下哪个海沟最深?
('B', '波多黎各海沟', 3